In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from matplotlib import pyplot as plt
from scipy.optimize import fsolve

In [2]:
#-------------------------------Find k----------------------------------------------
E = 6e04
ko = 1e-04
R = 8.314
To = 1140
def find_k(T):
    k = ko*np.exp(-E/(R*T))
    return k
k = find_k(To)

In [3]:
Fs = 1.26
V = 1
#---------------------Steady state outlet concentrations-----------------------------
Cas = 82.45
Cbs = 247.36
Ccs = 352.76
#---------------------Steady state inlet concentrations-----------------------------
Caos = 258.83
Cbos = 776.5
Ccos = 0

Kp = 1/12
tau = 10
ksi = 3
zeta = 3

Ccv = 0.09662
deltaP_cv = 2
rho = 8.81609
SG = rho/1000
Pso = 9
xs = 1/12*Pso

## Differential equations
The differential equations used to calculate the transfer functions are given below. 

Note that A is $N_2$, B is $H_2$, and C is $NH_3$

$$
\begin{bmatrix}
\frac{dC_A'}{dt} \\
\frac{dC_B'}{dt} \\
\frac{dC_C'}{dt}
\end{bmatrix}
=
\begin{bmatrix}
(-\frac{Fs}{V} - kC_{Bs}^3) & (-3kC_{As} C_{Bs}^2) & 0 \\
(-3kC_{Bs}^3) & (-\frac{Fs}{V} - 9kC_{As}C_{Bs}^2) & 0 \\
(2kC_{Bs}^3) & (6kC_{as}C_{Bs}^2) & (-\frac{Fs}{V})
\end{bmatrix}
\cdot
\begin{bmatrix}
C_A' \\
C_B' \\
C_C'
\end{bmatrix}
+
\begin{bmatrix}
\frac{C_{Ao}-C_A}{V} \\
\frac{C_{Bo}-C_B}{V} \\
0
\end{bmatrix}
\cdot
\begin{bmatrix}
F' 
\end{bmatrix}
\begin{bmatrix}
\frac{Fs}{V} \\
0 \\
0
\end{bmatrix}
\cdot
\begin{bmatrix}
C_{Ao}' 
\end{bmatrix}
$$



In [4]:
#-------------------------------A Matrix----------------------------------------------
a11 = -Fs/V-k*Cbs**3
a12 = -3*k*Cas*Cbs**2
a13 = 0
a21 = -3*k*Cbs**3
a22 = -Fs/V-9*k*Cas*Cbs**2
a23 = 0
a31 = 2*k*Cbs**3
a32 = 6*k*Cas*Cbs**2
a33 = -Fs/V
#---------------------B Matrix (coefficients of MV)-----------------------------------
b1 = (Caos-Cas)/V
b2 = (Cbos-Cbs)/V
b3 = -Ccs/V
#---------------------E Matrix (coefficients of DV)-----------------------------------
e1 = Fs/V
e2 = 0
e3 = 0

## Finding Gp(s)

In [5]:
A = sp.Matrix([[a11,a12,a13],[a21,a22,a23],[a31,a32,a33]])    #Coefficients of Ca, Cb, Cc
B = sp.Matrix([[b1],[b2],[b3]])                               #Coefficients of MV
E = sp.Matrix([[e1],[e2],[e3]])                               #Coefficients of DV
C = sp.Matrix([[0,0,1]])                                      #Indicates controlled variable is Cc
I = sp.eye(A.shape[0])

In [6]:
Cc,F,s = sp.symbols('Cc,F,s')
Gp1 = C*(s*I - A).inv()*B
Gp1 = sp.nsimplify(Gp1[0,0]).simplify()
display(sp.Eq(Cc/F,Gp1))

Eq(Cc/F, 26457*(-269586130178937*s**2 - 679357048050910*s - 427994940272065)/(25*(808758390536811*s**3 + 11778044077993900*s**2 + 25828716614095900*s + 15463181029766200)))

The transfer function above relates the outlet concentration of Ammonia ($C_c$) to the flow rate ($F$).

We now need to find $\frac{F}{P_s}$ so that we can multiply $\frac{F}{P_s}$ by the transfer function $\frac{C_c}{F}$ (above) in order to get the overall transfer function $\frac{C_c}{P_s}$.


### Method
To do this we begin by finding x/Ps. That is, how does the stem position x change with the pressure supplied. 

In [7]:
d11 = 0
d12 = 1
d21 = -1/tau**2
d22 = -2*ksi/tau

f1 = 0
f2 = Kp/tau**2

In [8]:
A2 = sp.Matrix([[d11,d12],[d21,d22]])
B2 = sp.Matrix([[f1],[f2]])
C2 = sp.Matrix([[1,0]])
I2 = sp.eye(A2.shape[0])

In [9]:
x,Ps,s = sp.symbols('x,Ps,s')
Gp2 = C2*(s*I2 - A2).inv()*B2
Gp2 = sp.nsimplify(Gp2[0,0]).simplify()
display(sp.Eq(x/Ps,Gp2))

Eq(x/Ps, 1/(12*(100*s**2 + 60*s + 1)))

### The next step

Now that we have $\frac{x}{P_s}$, we can find $\frac{F}{P_s}$. We know that after linearizing $f(x)$, $F' = k_1 \cdot x'$ where $k_1$ is some constant.

Re-arranging, we get that $x' = \frac{F'}{k_1}$.

We can substitute that into the equation above and find that $\frac{F'}{P_s} = k_1 \cdot \frac{x}{P_s}$.

#### F
$$F = C_{cv} \cdot \sqrt{x} \cdot \sqrt{\frac{\Delta P_{cv}}{SG}}$$

After linearisation:
$$F = C_{cv} \cdot \sqrt{\frac{\Delta P_{cv}}{SG}} \cdot \left(\sqrt{x_s} + \frac{x'}{2\sqrt{x_s}}\right)$$

#### k1

After expressing in terms of deviation variables, we find that k1 is given by:

$$k_1 = \frac{C_{cv}}{2} \cdot \sqrt{\frac{\Delta P_{cv}}{x_s \cdot SG}}$$

Substituting in the numerical values we know, we can multiply $k_1$ by the transfer function $\frac{x}{P_s}$ to obtain a new transfer function $\frac{F}{P_s}$ shown below.

In [10]:
F,Ps,s = sp.symbols('F,Ps,s')
Gp3 = Gp2*(Ccv/2*(deltaP_cv/xs/SG)**0.5) 
display(sp.Eq(F/Ps,Gp3))

Eq(F/Ps, 0.0700167825648624/(100*s**2 + 60*s + 1))

In [64]:
Gp = Gp1*Gp3

Cc,Ps,s = sp.symbols('Cc,Ps,s')
display(sp.Eq(Cc/Ps,Gp))
orderGp = sp.degree(sp.denom(Gp), s)
print("Order of Gp:", orderGp)

# Find the zeros and poles
zerosGp = sp.solve(sp.numer(Gp), s)  # Zeros
polesGp = sp.solve(sp.denom(Gp), s)  # Poles

# Numerical approximation of poles

numerical_polesGp = [sp.N(pole) for pole in polesGp]

nzerosGp=[sp.N(zero) for zero in zerosGp]
Gprz = [round(zero, 2) for zero in zerosGp]
Gprp = [round(pole, 2) for pole in numerical_polesGp]
print('Gprz',Gprz)
print('Gprp',Gprp)
print('ZerosGp',nzerosGp)
print("PolesGp:", numerical_polesGp)

polesGp = sp.solve(sp.denom(Gp), s)

# Numerical values of poles (taking only the real part)

print(numerical_polesGp)
# Factorized form of Gp
numeratorGp = 1
denominatorGp = 1

# Multiply the factors for zeros
for zero in Gprz:
    numeratorGp *= (s - zero)

# Multiply the factors for poles
for pole in Gprp:
    denominatorGp *= (s - pole)

factorized_Gp = numeratorGp / denominatorGp

# Simplify and evaluate the factorized expression
factorized_Gp_simplified = sp.simplify(factorized_Gp)
fGp = sp.N(factorized_Gp_simplified)


print("Factorized Gd (Simplified and Numeric):", fGp)

display(sp.Eq(Cc/C_Ao,fGp))

Eq(Cc/Ps, 74.0973606527426*(-269586130178937*s**2 - 679357048050910*s - 427994940272065)/((100*s**2 + 60*s + 1)*(808758390536811*s**3 + 11778044077993900*s**2 + 25828716614095900*s + 15463181029766200)))

Order of Gp: 5
Gprz [-1.26, -1.26]
Gprp [-0.58, -0.02, -1.26, -1.26, -12.04]
ZerosGp [-1.26000009575580, -1.25999990424416]
PolesGp: [-0.582842712474619, -0.0171572875253810, -1.26 + 4.5176959506e-8*I, -1.26 - 4.5176959506e-8*I, -12.0431182511458]
[-0.582842712474619, -0.0171572875253810, -1.26 + 4.5176959506e-8*I, -1.26 - 4.5176959506e-8*I, -12.0431182511458]
Factorized Gd (Simplified and Numeric): 1.0/((s + 0.02001953125)*(s + 0.580078125)*(s + 12.0400390625))


Eq(Cc/C_Ao, 1.0/((s + 0.02001953125)*(s + 0.580078125)*(s + 12.0400390625)))

## Finding Gd(s)

All that remains to find Gd(s) is to switch out the B matrix with the E matrix defined before to find the contribution made by the disturbance variable.

In [46]:
Cc,C_Ao,s = sp.symbols('Cc,C_Ao,s')
Gd = C*(s*I - A).inv()*E
Gd = sp.nsimplify(Gd[0,0]).simplify()
display(sp.Eq(Cc/C_Ao,Gd))
order = sp.degree(sp.denom(Gd), s)
print("Order:", order)

# Find the zeros and poles
zeros = sp.solve(sp.numer(Gd), s)  # Zeros
poles = sp.solve(sp.denom(Gd), s)  # Poles
poles = sp.solve(sp.denom(Gd), s)
# Numerical approximation of poles

numerical_poles = [sp.N(pole) for pole in poles]

nzeros=[sp.N(zero) for zero in zeros]
print('Zeros',nzeros)
print("Poles:", numerical_poles)
Gd2=(s+1.26)/(((s+1.26)**2)*(s+12.04))
poles = sp.solve(sp.denom(Gd), s)

# Numerical values of poles (taking only the real part)
numerical_poles = [sp.re(complex(p)) for p in poles]
print(numerical_poles)
# Factorized form of Gd
numerator = 1
denominator = 1

# Multiply the factors for zeros
for zero in nzeros:
    numerator *= (s - zero)

# Multiply the factors for poles
for pole in numerical_poles:
    denominator *= (s - pole)

factorized_Gd = numerator / denominator

# Simplify and evaluate the factorized expression
factorized_Gd_simplified = sp.simplify(factorized_Gd)
fGd = sp.N(factorized_Gd_simplified)


print("Factorized Gd (Simplified and Numeric):", fGd)

display(sp.Eq(Cc/C_Ao,fGd))

Eq(Cc/C_Ao, 252*(109015022377282*s + 137358928195375)/(5*(808758390536811*s**3 + 11778044077993900*s**2 + 25828716614095900*s + 15463181029766200)))

Order: 3
Zeros [-1.26000000000000]
Poles: [-1.26 + 4.5176959506e-8*I, -1.26 - 4.5176959506e-8*I, -12.0431182511458]
[-1.26000000000000, -1.26000000000000, -12.0431182511458]
Factorized Gd (Simplified and Numeric): (0.629881582262535*s + 0.793650793650792)/((0.793650793650794*s + 1.0)**2*(s + 12.0431182511458))


Eq(Cc/C_Ao, (0.629881582262535*s + 0.793650793650792)/((0.793650793650794*s + 1.0)**2*(s + 12.0431182511458)))

# Task 2: Block Diagram

![Block Diagram](Block_Diagram.png)

# Task 3: Comparison of numerical methods and linearized model

In [ ]:
def rates(var):
    Ca,Cb,To = var
    ra = -find_k(To)*Ca*Cb**3
    rb = 3*ra
    rc= -2*ra
    return [ra,rb,rc]


In [ ]:
def flow_valve(x):
    Fo = Ccv*(x)**0.5*(deltaP_cv/SG)**0.5
    return Fo


In [ ]:
def ManiVar(t, delta_Ps):
    return Pso if t<step_time else Pso + delta_Ps


In [ ]:
def diff(t,var,MV):
    x,x2,Ca,Cb,Cc = var
    
    #Rates calculation
    ra,rb,rc = rates([Ca,Cb,To])
    
    #Valve equations
    dxdt = x2
    dx2dt = (-2*zeta/tau)*x2 - (1/tau**2)*x +(Kp/tau**2)*MV
    Fo = flow_valve(x)
    
    #Mass Balance, Assuming density remains constant
    F1 = Fo
    
    #Component balances
    dCadt = (Fo*Caos - F1*Ca +ra*V)/V
    dCbdt = (Fo*Cbos - F1*Cb +rb*V)/V
    dCcdt = (Fo*Ccos - F1*Cc +rc*V)/V
    
    return np.array([dxdt,dx2dt,dCadt,dCbdt,dCcdt])
   

In [ ]:
def steady(var):
    return diff(0,var,Pso)

In [ ]:
def Euler(t,dt,var,MV):
    de_var = diff(t,var,MV)
    var = de_var*dt + var
    return var

In [ ]:
def Euler_Integration(tspan,dt, delta_Ps,ss_var):
    Values = []
    var = ss_var
    for i,t in enumerate(tspan):
        MV = ManiVar(t, delta_Ps)
        var = Euler(t,dt,var,MV)
        Values.append(var)
    return Values

In [ ]:
#Plotting
ss_var = fsolve(steady,[0.5,0,80,250,350])

time = (0,500)
tspan = np.linspace(*time,30000)
dt = tspan[1]

step_time = 10
deltaP_step_change_1 = 0.25
deltaP_step_change_2 = 4
Values_1 = Euler_Integration(tspan,dt, deltaP_step_change_1,ss_var)
Values_2 = Euler_Integration(tspan,dt, deltaP_step_change_2,ss_var)

Cc_vals_1 = np.array([Values_1[i][-1] for i in range(len(Values_1))])
Cc_vals_2 = np.array([Values_2[i][-1] for i in range(len(Values_2))])

data_1 = np.matrix(pd.read_csv("0.25 psi step data.csv"))
data_2 = np.matrix(pd.read_csv("4 psi step data.csv"))

times_scilab = data_1[:,0]

Cc_vals_1_scilab = data_1[:,1]
Cc_vals_2_scilab = data_2[:,1]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

axs[0].plot(tspan,Cc_vals_1 - Ccs,label="Euler method")
axs[0].plot(times_scilab,Cc_vals_1_scilab,label="Linearized transfer function")
axs[0].set_title("Graph showing $C_{NH_3}$ for a step change in Ps of 0.25 psi")
axs[0].set_xlabel("Time (s)")
axs[0].set_ylabel("Concentration (mol/$m^3$)")
axs[0].legend(loc="best")

axs[1].plot(tspan,Cc_vals_2 - Ccs,label="Euler method")
axs[1].plot(times_scilab,Cc_vals_2_scilab,label="Linearized transfer function")
axs[1].set_title("Graph showing $C_{NH_3}$ for a step change in Ps of 4 psi")
axs[1].set_xlabel("Time (s)")
axs[1].set_ylabel("Concentration (mol/$m^3$)")
axs[1].legend(loc="best")

plt.tight_layout()
plt.show()